In [1]:
import pandas as pd
import platform
import tensorflow as tf

In [ ]:
LABEL_COLUMN = 'label'
NUMERIC_COLUMNS = [
  'user_age',
  'item_age',
]
CATEGORICAL_COLUMNS = [
  'user_id',
  'item_id',
  'user_topic',
  'item_topic',
]
HASH_BUCKET_SIZES = {
  'user_id': 400,
  'item_id': 2000,
  'user_topic': 10,
  'item_topic': 10,
}
EMBEDDING_DIMENSIONS = {
    'user_id': 8,
    'item_id': 16,
    'user_topic': 3,
    'item_topic': 3,
}
 
def get_wide_and_deep_columns():
    wide_columns, deep_columns = [], []
    
    # embedding columns
    for column_name in CATEGORICAL_COLUMNS:
        categorical_column = tf.feature_column.categorical_column_with_identity(
                              column_name, num_buckets=HASH_BUCKET_SIZES[column_name])
        wrapped_column = tf.feature_column.embedding_column(
                              categorical_column,
                              dimension=EMBEDDING_DIMENSIONS[column_name],
                              combiner='mean')
        wide_columns.append(categorical_column)
        deep_columns.append(wrapped_column)
        
    # age columns and cross columns
    user_age = tf.feature_column.numeric_column("user_age", shape=(1,), dtype=tf.float32)
    item_age = tf.feature_column.numeric_column("item_age", shape=(1,), dtype=tf.float32)       
    user_age_buckets = tf.feature_column.bucketized_column(user_age, boundaries=[18, 35])
    item_age_buckets = tf.feature_column.bucketized_column(item_age, boundaries=[18, 35])
    age_crossed = tf.feature_column.crossed_column([user_age_buckets, item_age_buckets], 9)
    wide_columns.extend([user_age_buckets, item_age_buckets, age_crossed])
    deep_columns.extend([user_age, item_age])

    # topic columns and cross columns
    user_topic = tf.feature_column.categorical_column_with_identity(
        "user_topic", num_buckets=HASH_BUCKET_SIZES["user_topic"])
    item_topic = tf.feature_column.categorical_column_with_identity(
         "item_topic", num_buckets=HASH_BUCKET_SIZES["item_topic"])       
    topic_crossed = tf.feature_column.crossed_column([user_topic, item_topic], 30)
    wide_columns.append(topic_crossed)
    return wide_columns, deep_columns

In [ ]:
linear_model = LinearModel()
dnn_model = keras.Sequential([keras.layers.Dense(units=64),
                             keras.layers.Dense(units=1)])
combined_model = WideDeepModel(linear_model, dnn_model)
combined_model.compile(optimizer=['sgd', 'adam'], 'mse', ['mse'])
# define dnn_inputs and linear_inputs as separate numpy arrays or
# a single numpy array if dnn_inputs is same as linear_inputs.
combined_model.fit([linear_inputs, dnn_inputs], y, epochs)
# or define a single `tf.data.Dataset` that contains a single tensor or
# separate tensors for dnn_inputs and linear_inputs.
dataset = tf.data.Dataset.from_tensors(([linear_inputs, dnn_inputs], y))
combined_model.fit(dataset, epochs)

In [ ]:
linear_model = LinearModel()
linear_model.compile('adagrad', 'mse')
linear_model.fit(linear_inputs, y, epochs)
dnn_model = keras.Sequential([keras.layers.Dense(units=1)])
dnn_model.compile('rmsprop', 'mse')
dnn_model.fit(dnn_inputs, y, epochs)
combined_model = WideDeepModel(linear_model, dnn_model)
combined_model.compile(optimizer=['sgd', 'adam'], 'mse', ['mse'])
combined_model.fit([linear_inputs, dnn_inputs], y, epochs)